In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from pathlib import Path

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

SEED = 42
IMG_SIZE = (160, 260)
BATCH_SIZE = 16
EPOCHS = 10

BASE_DIR = Path.cwd().parent   # pastikan kamu run notebook dari root project
TRAIN_DIR = BASE_DIR / "dataset_split" / "train"
VAL_DIR   = BASE_DIR / "dataset_split" / "val"
TEST_DIR  = BASE_DIR / "dataset_split" / "test"

# CNN base cukup rescale 0-1 + augmentasi ringan
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

print("BASE_DIR:", BASE_DIR)
print("TRAIN_DIR exists?", TRAIN_DIR.exists(), TRAIN_DIR)
print("VAL_DIR exists?", VAL_DIR.exists(), VAL_DIR)
print("TEST_DIR exists?", TEST_DIR.exists(), TEST_DIR)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode="categorical", seed=SEED, shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    VAL_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode="categorical", seed=SEED, shuffle=False
)

test_gen = test_datagen.flow_from_directory(
    TEST_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode="categorical", seed=SEED, shuffle=False
)

num_classes = train_gen.num_classes
class_names = list(train_gen.class_indices.keys())
print("Classes:", class_names)


c:\Users\yunit\OneDrive\Desktop\Documents\UAP_sem7\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


BASE_DIR: c:\Users\yunit\OneDrive\Desktop\Documents\UAP_sem7
TRAIN_DIR exists? True c:\Users\yunit\OneDrive\Desktop\Documents\UAP_sem7\dataset_split\train
VAL_DIR exists? True c:\Users\yunit\OneDrive\Desktop\Documents\UAP_sem7\dataset_split\val
TEST_DIR exists? True c:\Users\yunit\OneDrive\Desktop\Documents\UAP_sem7\dataset_split\test
Found 4196 images belonging to 4 classes.
Found 900 images belonging to 4 classes.
Found 904 images belonging to 4 classes.
Classes: ['dark', 'light', 'mid-dark', 'mid-light']


In [2]:
from tensorflow.keras import layers, models

model_cnn = models.Sequential([
    layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.GlobalAveragePooling2D(),   # ✅ ganti Flatten
    layers.Dropout(0.4),
    layers.Dense(128, activation="relu"),
    layers.Dense(num_classes, activation="softmax")
])

model_cnn.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


model_cnn.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 158, 258, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 79, 129, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 77, 127, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 38, 63, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 36, 61, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 18, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,276 (430.77 KB)

 Trainable params: 110,276 (430.77 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from pathlib import Path

MODELS_DIR = BASE_DIR / "models"
MODELS_DIR.mkdir(exist_ok=True)

callbacks = [
    ModelCheckpoint(
        filepath=str(MODELS_DIR / "cnn_base.h5"),
        monitor="val_accuracy",
        save_best_only=True,
        mode="max",
        verbose=1
    ),
    EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1)
]

history = model_cnn.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)


Epoch 1/10
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3768 - loss: 1.2401
Epoch 1: val_accuracy improved from None to 0.54222, saving model to c:\Users\yunit\OneDrive\Desktop\Documents\UAP_sem7\models\cnn_base.h5



Epoch 1: finished saving model to c:\Users\yunit\OneDrive\Desktop\Documents\UAP_sem7\models\cnn_base.h5
263/263 ━━━━━━━━━━━━━━━━━━━━ 735s 3s/step - accuracy: 0.4519 - loss: 1.1225 - val_accuracy: 0.5422 - val_loss: 0.9675 - learning_rate: 0.0010
Epoch 2/10
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5594 - loss: 0.9491
Epoch 2: val_accuracy improved from 0.54222 to 0.56444, saving model to c:\Users\yunit\OneDrive\Desktop\Documents\UAP_sem7\models\cnn_base.h5



Epoch 2: finished saving model to c:\Users\yunit\OneDrive\Desktop\Documents\UAP_sem7\models\cnn_base.h5
263/263 ━━━━━━━━━━━━━━━━━━━━ 533s 2s/step - accuracy: 0.5651 - loss: 0.9253 - val_accuracy: 0.5644 - val_loss: 0.9414 - learning_rate: 0.0010
Epoch 3/10
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6263 - loss: 0.8399
Epoch 3: val_accuracy improved from 0.56444 to 0.56889, saving model to c:\Users\yunit\OneDrive\Desktop\Documents\UAP_sem7\models\cnn_base.h5



Epoch 3: finished saving model to c:\Users\yunit\OneDrive\Desktop\Documents\UAP_sem7\models\cnn_base.h5
263/263 ━━━━━━━━━━━━━━━━━━━━ 532s 2s/step - accuracy: 0.6261 - loss: 0.8430 - val_accuracy: 0.5689 - val_loss: 0.9397 - learning_rate: 0.0010
Epoch 4/10
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6205 - loss: 0.8269
Epoch 4: val_accuracy improved from 0.56889 to 0.62444, saving model to c:\Users\yunit\OneDrive\Desktop\Documents\UAP_sem7\models\cnn_base.h5



Epoch 4: finished saving model to c:\Users\yunit\OneDrive\Desktop\Documents\UAP_sem7\models\cnn_base.h5
263/263 ━━━━━━━━━━━━━━━━━━━━ 439s 2s/step - accuracy: 0.6251 - loss: 0.8194 - val_accuracy: 0.6244 - val_loss: 0.8599 - learning_rate: 0.0010
Epoch 5/10
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6468 - loss: 0.8054
Epoch 5: val_accuracy improved from 0.62444 to 0.64000, saving model to c:\Users\yunit\OneDrive\Desktop\Documents\UAP_sem7\models\cnn_base.h5



Epoch 5: finished saving model to c:\Users\yunit\OneDrive\Desktop\Documents\UAP_sem7\models\cnn_base.h5
263/263 ━━━━━━━━━━━━━━━━━━━━ 443s 2s/step - accuracy: 0.6463 - loss: 0.8068 - val_accuracy: 0.6400 - val_loss: 0.8185 - learning_rate: 0.0010
Epoch 6/10
257/263 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.6490 - loss: 0.7939

In [ ]:
plt.figure()
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.title("Loss (CNN Base)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.figure()
plt.plot(history.history["accuracy"], label="train_acc")
plt.plot(history.history["val_accuracy"], label="val_acc")
plt.title("Accuracy (CNN Base)")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
# Prediksi di test set
test_gen.reset()
pred_probs = model_cnn.predict(test_gen)
y_pred = np.argmax(pred_probs, axis=1)
y_true = test_gen.classes

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)


In [ ]:
plt.figure()
plt.imshow(cm)
plt.title("Confusion Matrix (CNN Base)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.xticks(range(len(class_names)), class_names, rotation=45)
plt.yticks(range(len(class_names)), class_names)
plt.colorbar()
plt.show()
